In [23]:
from scipy.spatial import distance
import numpy as np
import json
import os
import math
import csv

In [19]:
def jaccard_similarity(list1, list2):
    intersection = len(set(list1).intersection(list2)) 
    union = len(list1 + list2) - intersection 
    return intersection / union 

In [29]:
#Loading Json File
def getMetric(json_file_1, json_file_2):
# json_file_1 = "metaanalysis/AB_Pas_N_Tablet/AB_Pas_N_Tablet_1.json"
# json_file_2 = "metaanalysis/AB_Pas_N_Tablet/AB_Pas_N_Tablet_2.json"
    with open(json_file_1, 'r') as f1:
        json_data1 = json.load(f1)

    text_dict1 = {
        "Word List": json_data1['Text']["Word List"]
    }
    colour_features_dict1 = {
        "Colour Features": json_data1["Color Moments"]["Colour Features"]
    }
    texture_dict1 =  json_data1["Texture"]
    shape_dict1 =  json_data1["Shape"]
    pattern_dict1 = json_data1["Pattern"]

    with open(json_file_2, 'r') as f2:
        json_data2 = json.load(f2)
    text_dict2 = {
        "Word List": json_data2['Text']["Word List"]
    }
    colour_features_dict2 = {
        "Colour Features": json_data2["Color Moments"]["Colour Features"]
    }
    texture_dict2 =  json_data2["Texture"]
    shape_dict2 =  json_data2["Shape"]
    pattern_dict2 = json_data2["Pattern"]

    #word list  using Jaccardian Distance
    word_list1 = text_dict1['Word List']
    word_list2 = text_dict2['Word List']
    dw = 0
    for l in word_list1:
        dw = 1-jaccard_similarity(word_list1, word_list2)

    #color list using Euclidean Distance
    color_list1 = colour_features_dict1['Colour Features']
    color_list2 = colour_features_dict2['Colour Features']
    dc=distance.euclidean(color_list1,color_list2)

    #texture list using Euclidean Distance
    texture_list1 = list(texture_dict1.values())
    texture_list2 = list(texture_dict2.values())
    dtx =distance.euclidean(texture_list1,texture_list2)

    #shape list using Euclidean Distance
    shape_list1 = list(shape_dict1.values())
    shape_list2 = list(shape_dict2.values())
    ds = distance.euclidean(shape_list1,shape_list2)

    #pattern list using Euclidean Distance
    pattern_list1 = list(pattern_dict1.values())
    pattern_list2 = list(pattern_dict2.values())
    dp = distance.euclidean(pattern_list1,pattern_list2)

    d = math.sqrt((dw**2)+(dc**2)+(dtx**2)+(ds**2)+(dp**2))

    return d




In [30]:
getMetric("metaanalysis/AB_Pas_N_Tablet/AB_Pas_N_Tablet_1.json", "metaanalysis/AB_Pas_N_Tablet/AB_Pas_N_Tablet_2.json")

581.7612416559077

In [31]:
import os
import csv

# define the paths to the metaanalysis folder and the output CSV file
metaanalysis_path = "metaanalysis"
output_csv_path = "scores.csv"

# define the function to calculate the score
results = []

# iterate through the subfolders
for subdir in os.listdir(metaanalysis_path):
    subdir_path = os.path.join(metaanalysis_path, subdir)
    if os.path.isdir(subdir_path):
        # create a list to store the scores for this subfolder
        subfolder_scores = [subdir]

        # iterate through the json files in the subfolder
        json_files = [f for f in os.listdir(subdir_path) if f.endswith('.json')]
        for i in range(len(json_files)):
            for j in range(i + 1, len(json_files)):
                # calculate the score for this pair of json files
                score = getMetric(os.path.join(subdir_path, json_files[i]), os.path.join(subdir_path, json_files[j]))

                # append the score to the subfolder_scores list
                subfolder_scores.append(score)

        # append the subfolder_scores list to the results list
        results.append(subfolder_scores)

# write the results to the output CSV file
with open(output_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Medicine Name', 'value_1', 'value_2', ...])
    for row in results:
        writer.writerow(row)
